# Recurrent Convolutional Neural Network


## Importing libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms, datasets
import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


## Load data

In [10]:
data=np.load("./data_1.npy")
data.shape #(num_event, 500,500, num_image)

(5, 500, 500, 30)

In [31]:
data=pd.read_csv("15_first_events.csv")


#maybe we will need to use that
transform = transforms.Compose([
    transforms.ToTensor(),           # Convert to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize if needed
])
#Specify the path to your custom dataset
custom_dataset_path = '/path/to/your/dataset' #normally ./data/0/ and ./data/1/

# Load the custom dataset using ImageFolder
custom_dataset = ImageFolder(root=custom_dataset_path, transform=transform)

# Create a DataLoader for iterating over batches
custom_dataloader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Example: Iterate over batches
for images, labels in custom_dataloader:
    # Your custom processing or training logic here
    pass

## Implementing RCNN

Exemple from the paper

In [6]:
class RCNN(torch.nn.Module):
    def __init__(self):
        super(RCNN, self).__init__()
        
        self.input_layer=torch.nn.Linear(in_features=500*500*15, out_features=500*500*15)

        self.norm=torch.nn.BatchNorm1d()#not sure this is sufficent for batch renormalization

        #Three convolutionnal and batch renormalization
        self.conv1=torch.nn.Conv1d(in_channels=15, out_channels=64, kernel_size=3, stride=1) #need padding of one normally but not precised
        self.relu1=torch.nn.ReLU()
        self.norm1=torch.nn.BatchNorm1d()

        self.conv2=torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu2=torch.nn.ReLU()
        self.norm2=torch.nn.BatchNorm1d()

        self.conv3=torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu3=torch.nn.ReLU()
        self.norm3=torch.nn.BatchNorm1d()

        #First max pooling
        self.max1=torch.nn.MaxPool1d(kernel_size=2,stride=2)

        #Three convolutionnal and batch renormalization
        self.conv4=torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu4=torch.nn.ReLU()
        self.norm4=torch.nn.BatchNorm1d()

        self.conv5=torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu5=torch.nn.ReLU()
        self.norm5=torch.nn.BatchNorm1d()

        self.conv6=torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu6=torch.nn.ReLU()
        self.norm6=torch.nn.BatchNorm1d()

        #Second maxpool
        self.max2=torch.nn.MaxPool1d(stride=2,kernel_size=2)

        #Flatten
        self.flat1=torch.nn.Flatten()

        #fully connected with dropout
        self.lin=torch.nn.Linear(in_features=2304, out_features=1024)
        self.drop=torch.nn.Dropout(p=0.5)

        #LSTM
        self.lstm=torch.nn.LSTM(input_size=1024,output_size=512, num_layers=1)

        #output
        self.output_layer=torch.nn.Softmax(dim=1)
        
    def forward(self,x):
        x=self.input_layer(x)
        x=self.norm(x)#not sure this is sufficent for batch renormalization

        #Three convolutionnal and batch renormalization
        x=self.conv1(x) 
        x=self.relu1(x)
        x=self.norm1(x)

        x=self.conv2(x)
        x=self.relu2(x)
        x=self.norm2(x)

        x=self.conv3(x)
        x=self.relu3(x)
        x=self.norm3(x)

        #First max pooling
        x=self.max1(x)

        #Three convolutionnal and batch renormalization
        x=self.conv4(x)
        x=self.relu4(x)
        x=self.norm4(x)

        x=self.conv5(x)
        x=self.relu5(x)
        x=self.norm5(x)

        x=self.conv6(x)
        x=self.relu6(x)
        x=self.norm6(x)

        #Second maxpool
        x=self.max2(x)

        #Flatten
        x=self.flat1(x)

        #fully connected with dropout
        x=self.lin(x)
        x=self.drop(x)

        #LSTM
        x=self.lstm(x)

        #output
        x=self.output_layer(x)
        
        return x

Another way to do it:

In [ ]:
model2=torch.nn.Sequential(

    torch.nn.Linear(in_features=500*500*15, out_features=500*500*15),

    torch.nn.BatchNorm1d(),#not sure this is sufficent for batch renormalization-->

        #Three convolutionnal and batch renormalization
    torch.nn.Conv1d(in_channels=15, out_channels=64, kernel_size=3, stride=1), #need padding of one normally but not precised
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(),

    torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(),

    torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(),

        #First max pooling
    torch.nn.MaxPool1d(kernel_size=2,stride=2),

        #Three convolutionnal and batch renormalization
    torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(),

    torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(),

    torch.nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(),

        #Second maxpool
    torch.nn.MaxPool1d(stride=2,kernel_size=2),

        #Flatten
    torch.nn.Flatten(),

        #fully connected with dropout
    torch.nn.Linear(in_features=2304, out_features=1024),
    torch.nn.Dropout(p=0.5),

        #LSTM
    torch.nn.LSTM(input_size=1024,output_size=512, num_layers=1),

        #output
    torch.nn.Softmax(dim=1),

)

In [ ]:
#This is an exemple of batch renormalization
class BatchRenorm2d(torch.nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, r_d_max_inc_step=0.0001):
        super(BatchRenorm2d, self).__init__()

        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.r_d_max_inc_step = r_d_max_inc_step

        # Parameters
        self.weight = torch.nn.Parameter(torch.ones(1, num_features, 1, 1))
        self.bias = torch.nn.Parameter(torch.zeros(1, num_features, 1, 1))
        self.running_mean = torch.nn.Parameter(torch.zeros(1, num_features, 1, 1))
        self.running_var = torch.nn.Parameter(torch.ones(1, num_features, 1, 1))
        self.gamma = torch.nn.Parameter(torch.Tensor([1.0]))
        self.beta = torch.nn.Parameter(torch.Tensor([0.0]))

        # Batch renormalization parameters
        self.r_max = torch.nn.Parameter(torch.Tensor([3.0]))
        self.d_max = torch.nn.Parameter(torch.Tensor([5.0]))
        self.c_max = torch.nn.Parameter(torch.Tensor([5.0]))

    def forward(self, x):
        if self.training:
            mean = x.mean(dim=[0, 2, 3], keepdim=True)
            var = x.var(dim=[0, 2, 3], unbiased=False, keepdim=True)

            self.running_mean.mul_(1 - self.momentum)
            self.running_var.mul_(1 - self.momentum)
            self.running_mean.add_(mean * self.momentum)
            self.running_var.add_(var * self.momentum)

            r_d_max_step = self.r_d_max_inc_step * (self.r_max / 3.0)
            r_max = torch.clamp(self.r_max + r_d_max_step, min=1.0, max=3.0)
            d_max = torch.clamp(self.d_max + r_d_max_step, min=0.0, max=5.0)

            x = torch.nn.functional.batch_norm(
                x, self.running_mean, self.running_var, self.weight, self.bias,
                training=True, eps=self.eps
            )
            x = x * self.gamma + self.beta
            x = torch.nn.functional.batch_renorm(
                x, self.running_mean, self.running_var,
                weight=self.weight, bias=self.bias,
                running_mean=None, running_var=None,
                momentum=self.momentum, eps=self.eps,
                rmax=r_max.item(), dmax=d_max.item(), cmax=self.c_max.item()
            )
        else:
            x = torch.nn.functional.batch_norm(
                x, self.running_mean, self.running_var, self.weight, self.bias,
                training=False, eps=self.eps
            )
            x = x * self.gamma + self.beta

        return x

## Training script


In [ ]:
def train(model, criterion, dataset_train, dataset_test, optimizer, num_epochs, device):
    """
    @param model: torch.nn.Module
    @param criterion: torch.nn.modules.loss._Loss
    @param dataset_train: torch.utils.data.DataLoader
    @param dataset_test: torch.utils.data.DataLoader
    @param optimizer: torch.optim.Optimizer
    @param num_epochs: int
    """
    print("Starting training")
    for epoch in range(num_epochs):
        # Train an epoch
        model.train()
        for batch_x, batch_y in dataset_train:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        
            # Evaluate the network (forward pass)
            prediction = model(batch_x)
            loss = criterion(prediction, batch_y)

            # Compute the gradient
            optimizer.zero_grad()
            loss.backward()

            # Update the parameters of the model with a gradient step
            optimizer.step()
        

        # Test the quality on the test set
        model.eval()
        accuracies_test = []
        for batch_x, batch_y in dataset_test:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Evaluate the network (forward pass)
            prediction = model(batch_x)
            accuracies_test.append(accuracy(prediction, batch_y))

        print(
            "Epoch {} | Test accuracy: {:.5f}".format(
                epoch, sum(accuracies_test).item() / len(accuracies_test)
            )
        )

In [ ]:
num_epochs = 10
learning_rate = 1e-3
batch_size = 1000
model=RCNN()
criterion = (
    torch.nn.CrossEntropyLoss()
)  # this includes LogSoftmax which executes a logistic transformation

optimizer= torch.optim.AdamW(model.parameters(), lr=learning_rate)
train(model, criterion, dataset_train, dataset_test, optimizer, num_epochs)